# 🇧🇷 Script Python para Análise de Horários Disponíveis
Com base na estrutura do seu arquivo JSON aulas_coletadas.json, preparei um script Python, ideal para ser executado em um notebook (como Jupyter ou Google Colab), que permite analisar os horários e a disponibilidade de aulas sob vários ângulos.

Este script usa a biblioteca pandas para manipular os dados de forma eficiente e datetime para o processamento das datas e horários.

### 1. Preparação e Carregamento de Dados
Primeiro, você precisará de uma maneira de carregar seu JSON.

In [ ]:
import pandas as pd
import json
import os
from datetime import datetime, time, timedelta

# --- Configuração de Arquivos ---
# Caminhos fornecidos pelo usuário
ARQUIVO_AULAS = os.path.join('..', 'data', 'aulas_coletadas.json')
ARQUIVO_MAPA_TURMAS = os.path.join('..', 'data', 'mapa_turmas.json')
ARQUIVO_HORARIOS_SEMANAIS = os.path.join('..', 'data', 'horarios_semanais_oficial.json')

# Mapeamento para nomes de dias da semana em português
MAPA_DIA_INT_PT = {
    0: 'segunda-feira', 1: 'terça-feira', 2: 'quarta-feira', 
    3: 'quinta-feira', 4: 'sexta-feira', 5: 'sábado', 6: 'domingo'
}
MAPA_DIA_PT_INT = {v: k for k, v in MAPA_DIA_INT_PT.items()}


# --- 1. Carregamento e Pré-processamento de Dados ---

def carregar_dados():
    """Carrega os dados dos arquivos JSON com tratamento de erros."""
    data = {}
    
    try:
        with open(ARQUIVO_AULAS, 'r', encoding='utf-8') as f:
            data['df_aulas_raw'] = pd.DataFrame(json.load(f))
        
        with open(ARQUIVO_MAPA_TURMAS, 'r', encoding='utf-8') as f:
            data['mapa_turmas'] = json.load(f)
        
        # O arquivo horarios_semanais é uma lista com um objeto dentro
        with open(ARQUIVO_HORARIOS_SEMANAIS, 'r', encoding='utf-8') as f:
            data['dados_agenda_raw'] = json.load(f)[0]
        
        print("✅ Todos os arquivos carregados com sucesso.")
        return data

    except FileNotFoundError as e:
        print(f"❌ Erro: Arquivo não encontrado: {e.filename}")
        print("⚠️ Por favor, verifique se os caminhos dos arquivos estão corretos.")
        return None
    except json.JSONDecodeError:
        print("❌ Erro: O arquivo JSON está mal formatado.")
        return None
    except Exception as e:
        print(f"❌ Ocorreu um erro desconhecido durante o carregamento: {e}")
        return None


def preprocessar_aulas(df_aulas_raw, mapa_turmas):
    """Pré-processamento das aulas coletadas, criando colunas de tempo e mapeamento."""
    
    # Filtrar apenas aulas confirmadas e fazer uma cópia
    df = df_aulas_raw[df_aulas_raw['status'] == 'Aula confirmada'].copy()
    
    # 1. Aplica mapeamento de turma
    df['turma_curta'] = df['turma'].apply(lambda x: mapa_turmas.get(x, x))
    
    # 2. Processamento de data e horário
    df[['horario_inicio_str', 'horario_fim_str']] = df['horario'].str.split(' às ', expand=True)
    df['data_aula_dt'] = pd.to_datetime(df['dataAula'], format='%d/%m/%Y', errors='coerce')
    
    # 3. Cálculo da Duração (em minutos) - Correção aplicada
    def calcular_duracao(row):
        try:
            start_dt = datetime.strptime(row['horario_inicio_str'], '%H:%M').time()
            end_dt = datetime.strptime(row['horario_fim_str'], '%H:%M').time()
            
            start_datetime = datetime.combine(row['data_aula_dt'].date(), start_dt)
            end_datetime = datetime.combine(row['data_aula_dt'].date(), end_dt)
            
            # Trata casos onde o horário final é menor que o inicial (ex: 23:30 - 00:30)
            if end_datetime < start_datetime:
                 end_datetime += timedelta(days=1)
            
            return int((end_datetime - start_datetime).total_seconds() / 60)
        except:
            return 0

    df['duracao_minutos'] = df.apply(calcular_duracao, axis=1)
    
    # 4. Criação das chaves de comparação
    df['dia_semana'] = df['data_aula_dt'].dt.dayofweek
    df['horario_inicio'] = df['horario_inicio_str']
    df['chave_agendamento'] = df['turma_curta'] + '-' + df['dia_semana'].astype(str) + '-' + df['horario_inicio']
    
    return df.dropna(subset=['data_aula_dt']) # Remove linhas com erro no parse da data


def normalizar_agenda(dados_agenda_raw):
    """Transforma a estrutura complexa da agenda em um DataFrame simples."""
    agenda_normalizada = []
    
    for professor, prof_data in dados_agenda_raw['professores'].items():
        for turma_curta, comps in prof_data['turmas'].items():
            for componente, horarios in comps.items():
                for h in horarios:
                    if isinstance(h, dict) and 'label_horario' in h:
                        try:
                            start_time_str, end_time_str = h['label_horario'].replace(' ', '').split('-')
                            dia_semana_nome = h['dia_semana_nome'].lower()
                            
                            agenda_normalizada.append({
                                'professor': professor,
                                'turma_curta': turma_curta,
                                'componenteCurricular': componente,
                                'dia_semana_nome': dia_semana_nome,
                                'dia_semana': MAPA_DIA_PT_INT.get(dia_semana_nome),
                                'horario_inicio': start_time_str,
                                'horario_fim': end_time_str,
                            })
                        except Exception as e:
                            # Ignora linhas mal formatadas
                            # print(f"Aviso: Pulando item mal formatado na agenda: {h}. Erro: {e}") 
                            pass
    
    df_agenda = pd.DataFrame(agenda_normalizada)
    df_agenda['chave_agendamento'] = df_agenda['turma_curta'] + '-' + df_agenda['dia_semana'].astype(str) + '-' + df_agenda['horario_inicio']
    
    return df_agenda


# --- 2. Execução do Pré-processamento ---

dados = carregar_dados()

if dados:
    df_aulas = preprocessar_aulas(dados['df_aulas_raw'], dados['mapa_turmas'])
    df_agenda = normalizar_agenda(dados['dados_agenda_raw'])
    
    if df_aulas.empty or df_agenda.empty:
         print("❌ Aviso: Um dos DataFrames de análise está vazio após o pré-processamento.")
    else:
        print("✅ DataFrames prontos para análise.")

        # --- 3. Análises de Disponibilidade ---

        print("\n" + "="*80)
        print("                           📊 RESULTADOS DA ANÁLISE DE DISPONIBILIDADE")
        print("="*80)
        
        # 3.1. Análise de Frequência de Cumprimento da Agenda Fixa (Planejado vs. Coletado)
        
        # Calcular o período de coleta para referência de slots esperados
        min_date = df_aulas['data_aula_dt'].min()
        max_date = df_aulas['data_aula_dt'].max()
        total_dias_analisados = (max_date - min_date).days + 1
        total_semanas = total_dias_analisados // 7
        
        print(f"\n📅 Período coletado: {min_date.strftime('%d/%m/%Y')} a {max_date.strftime('%d/%m/%Y')} ({total_semanas} semanas de referência)")
        
        frequencia_cumprimento = df_aulas['chave_agendamento'].value_counts().reset_index()
        frequencia_cumprimento.columns = ['chave_agendamento', 'aulas_registradas_coletadas']

        df_analise = df_agenda.merge(frequencia_cumprimento, on='chave_agendamento', how='left').fillna(0)
        df_analise['aulas_registradas_coletadas'] = df_analise['aulas_registradas_coletadas'].astype(int)

        # O Slots_Esperados_Ref é uma estimativa; PRECISA do 'ano_letivo.json' para ser exato.
        df_analise['slots_esperados'] = total_semanas 
        
        df_analise['percentual_cumprimento'] = (df_analise['aulas_registradas_coletadas'] / df_analise['slots_esperados'] * 100).round(1)
        
        df_cumprimento = df_analise[[
            'professor', 'turma_curta', 'componenteCurricular', 
            'dia_semana_nome', 'horario_inicio', 'horario_fim', 
            'aulas_registradas_coletadas', 'slots_esperados', 
            'percentual_cumprimento'
        ]].rename(columns={
            'aulas_registradas_coletadas': 'Aulas_Coletadas',
            'slots_esperados': 'Slots_Esperados_Ref',
            'percentual_cumprimento': 'Cumprimento_%'
        })

        print("\n--- 📊 3.1. Cumprimento da Agenda Fixa (Planejado vs. Coletado) ---")
        print("Slots_Esperados_Ref = Total de semanas no período. Desvios (Cumprimento_% < 100) indicam aulas não registradas.")
        print(df_cumprimento.sort_values(by='Cumprimento_%', ascending=True).to_string())
        
        print("\n" + "-"*80)

        # 3.2. Análise por Componente Curricular (Total de Horas Coletadas)
        
        aulas_por_disciplina = df_aulas.groupby('componenteCurricular').agg(
            total_aulas=('turma_curta', 'count'),
            total_horas=('duracao_minutos', 'sum')
        ).sort_values(by='total_aulas', ascending=False)
        
        aulas_por_disciplina['total_horas'] = (aulas_por_disciplina['total_horas'] / 60).round(2)
        
        print("\n--- 📚 3.2. Distribuição de Aulas por Disciplina (Coletadas) ---")
        print("Total de aulas e horas ministradas por disciplina (aulas confirmadas).")
        print(aulas_por_disciplina)
        
        print("\n" + "-"*80)

        # 3.3. Identificação de Slots Vagos na Agenda (Oportunidades de Agendamento - "Furos")
        
        horarios_disponiveis = dados['dados_agenda_raw']['config']['horario_id_para_label']
        turmas_na_agenda = df_agenda['turma_curta'].unique()
        
        slots_possiveis = []
        for turma in turmas_na_agenda:
            for dia_int, dia_nome in MAPA_DIA_INT_PT.items():
                if dia_int < 5: # Considera apenas dias úteis
                    for horario_id, label_horario in horarios_disponiveis.items():
                        start_time_str, _ = label_horario.replace(' ', '').split('-')
                        slots_possiveis.append({
                            'turma_curta': turma,
                            'dia_semana_nome': dia_nome,
                            'label_horario': label_horario,
                            'chave_possivel': turma + '-' + str(dia_int) + '-' + start_time_str
                        })

        df_slots_possiveis = pd.DataFrame(slots_possiveis)
        slots_agendados = set(df_agenda['chave_agendamento'])

        df_slots_possiveis['agendado'] = df_slots_possiveis['chave_possivel'].apply(lambda x: x in slots_agendados)

        furos_na_agenda = df_slots_possiveis[df_slots_possiveis['agendado'] == False]
        df_furos = furos_na_agenda[['turma_curta', 'dia_semana_nome', 'label_horario']]
        
        print("\n--- 🎯 3.3. Slots Vagos na Agenda (Oportunidades de Agendamento) ---")
        print("Estes são horários que turmas específicas não têm aula agendada na grade fixa semanal (furos/disponibilidade total).")
        print(df_furos.sort_values(by=['turma_curta', 'dia_semana_nome']).to_string())
        
        print("\n" + "="*80)

### 2. Função de Pré-processamento e Limpeza
É crucial converter as colunas de data e horário para formatos que o Python e o Pandas possam manipular.

In [ ]:
def preprocessar_aulas(df_aulas_raw, mapa_turmas):
    """Pré-processamento das aulas coletadas (função anterior, simplificada)."""
    
    # Certifica-se de que a coluna existe antes de prosseguir
    if 'componenteCurricular' not in df_aulas_raw.columns:
        print("⚠️ ERRO DE COLUNA: 'componenteCurricular' não encontrada no JSON original.")
        # Se você souber o nome correto da coluna no seu JSON, ajuste aqui:
        # Ex: df_aulas_raw.rename(columns={'nome_correto': 'componenteCurricular'}, inplace=True)
        # Se não tiver essa coluna, a análise 3.2 não funcionará.
        # Por segurança, vamos apenas retornar um DataFrame vazio ou causar erro claro.
        raise KeyError("A coluna 'componenteCurricular' não foi encontrada no DataFrame de aulas brutas.")

    df = df_aulas_raw[df_aulas_raw['status'] == 'Aula confirmada'].copy()
    
    # Aplica mapeamento
    df['turma_curta'] = df['turma'].apply(lambda x: mapa_turmas.get(x, x))
    
    # Processamento de horário
    df[['horario_inicio', 'horario_fim']] = df['horario'].str.split(' às ', expand=True)
    
    # Processamento de data e dia da semana
    df['data_aula_dt'] = pd.to_datetime(df['dataAula'], format='%d/%m/%Y')
    df['dia_semana'] = df['data_aula_dt'].dt.dayofweek
    
    # Cria uma chave única para comparação com a agenda semanal (Dia + Hora + Turma)
    df['chave_agendamento'] = df['turma_curta'] + '-' + df['dia_semana'].astype(str) + '-' + df['horario_inicio']
    
    return df

### 3. Análises de Disponibilidade por Vários Ângulos
Agora, o script irá gerar as informações solicitadas.

#### 3.1. Visão Geral: Horários mais e menos Ocupados (por Dia da Semana e Hora)
Esta análise mostra a distribuição de aulas por dia da semana e o horário de início, ajudando a identificar picos e vales na agenda semanal.

In [ ]:
# Contar quantas vezes cada slot da agenda fixa FOI registrado no período coletado
frequencia_cumprimento = df_aulas['chave_agendamento'].value_counts().reset_index()
frequencia_cumprimento.columns = ['chave_agendamento', 'aulas_registradas_coletadas']

# Cruza a frequência de cumprimento com a agenda fixa
df_analise = df_agenda.merge(frequencia_cumprimento, on='chave_agendamento', how='left').fillna(0)
df_analise['aulas_registradas_coletadas'] = df_analise['aulas_registradas_coletadas'].astype(int)

# Contagem de Ocorrências: Quantas vezes aquele slot (Dia/Hora/Turma) deveria ter ocorrido
# *Se não tivéssemos o arquivo do ano letivo*, assumimos que cada slot deveria ter ocorrido
# uma vez por semana ao longo das 860+ aulas.

# Para simplificar sem o ano letivo, vamos analisar a média de registro:
total_dias_analisados = (df_aulas['data_aula_dt'].max() - df_aulas['data_aula_dt'].min()).days + 1
print(f"\nPeríodo coletado: {df_aulas['dataAula'].min()} a {df_aulas['dataAula'].max()} ({total_dias_analisados} dias)")

# Aulas_por_semana_min: Para ter uma referência, vamos calcular quantas vezes o slot deveria 
# ter ocorrido se considerarmos um período de 12 semanas (apenas um exemplo, ajuste necessário
# com base no 'ano letivo.json')
total_semanas = total_dias_analisados // 7
df_analise['slots_esperados'] = total_semanas # Assumindo 1 ocorrência por semana

df_analise['diferenca'] = df_analise['aulas_registradas_coletadas'] - df_analise['slots_esperados']
df_analise['percentual_cumprimento'] = (df_analise['aulas_registradas_coletadas'] / df_analise['slots_esperados'] * 100).round(1)

# Seleciona colunas de interesse
df_cumprimento = df_analise[[
    'professor', 'turma_curta', 'componenteCurricular', 
    'dia_semana_nome', 'horario_inicio', 'horario_fim', 
    'aulas_registradas_coletadas', 'slots_esperados', 
    'percentual_cumprimento'
]]
df_cumprimento.rename(columns={
    'aulas_registradas_coletadas': 'Aulas_Coletadas',
    'slots_esperados': 'Slots_Esperados_Ref',
    'percentual_cumprimento': 'Cumprimento_%'
}, inplace=True)

print("\n--- 📊 3.1. Cumprimento da Agenda Fixa (Planejado vs. Coletado) ---")
print("Slots Esperados é uma referência (total de semanas no período coletado) e PRECISA ser ajustado pelo 'ano letivo.json' para maior precisão.")
print(df_cumprimento.sort_values(by='Cumprimento_%', ascending=True).to_string())
#
#

#### 3.2. Disponibilidade por Componente Curricular (Disciplina)
Ajuda a ver quantas vezes cada disciplina é oferecida.

In [ ]:
## 3.2. Carga Horária Planejada por Professor

# O cálculo de duração da aula é necessário para a agenda, pois o label_horario não tem o campo de duração
def calcular_duracao_agenda(start_str, end_str):
    start = datetime.strptime(start_str, '%H:%M')
    end = datetime.strptime(end_str, '%H:%M')
    # Trata caso de virada de dia (se necessário)
    if end < start:
        end += timedelta(days=1)
    return (end - start).total_seconds() / 3600 # Duração em Horas

# Aplica o cálculo no DataFrame de análise
df_analise['duracao_horas'] = df_analise.apply(
    lambda row: calcular_duracao_agenda(row['horario_inicio'], row['horario_fim']), 
    axis=1
)

carga_planejada_professor = df_analise.groupby('professor')['duracao_horas'].sum().round(2).reset_index()
carga_planejada_professor.columns = ['Professor', 'Carga_Horária_Planejada_Horas']

print("\n--- 👨‍🏫 3.2. Carga Horária Semanal Planejada por Professor ---")
print(carga_planejada_professor.sort_values(by='Carga_Horária_Planejada_Horas', ascending=False).to_string())

In [ ]:
## 3.2. Análise por Componente Curricular (Disciplina)

aulas_por_disciplina = df_aulas.groupby('componenteCurricular').agg(
    total_aulas=('turma', 'count'),
    total_horas=('duracao_minutos', 'sum')
).sort_values(by='total_aulas', ascending=False)

# Converter total_horas de minutos para horas (para melhor visualização)
aulas_por_disciplina['total_horas'] = (aulas_por_disciplina['total_horas'] / 60).round(2)

print("\n--- 📚 3.2. Distribuição de Aulas por Disciplina ---")
print(aulas_por_disciplina)

#### 3.3. Disponibilidade por Turma
Mostra a carga horária de aulas para cada turma.

In [ ]:
## 3.3. Análise por Turma

aulas_por_turma = df_aulas.groupby('turma').agg(
    total_aulas=('componenteCurricular', 'count'),
    total_horas=('duracao_minutos', 'sum'),
    primeira_aula=('data_aula_dt', 'min'),
    ultima_aula=('data_aula_dt', 'max')
).sort_values(by='total_aulas', ascending=False)

# Converter total_horas de minutos para horas
aulas_por_turma['total_horas'] = (aulas_por_turma['total_horas'] / 60).round(2)

print("\n--- 🏫 3.3. Carga Horária de Aulas por Turma ---")
print(aulas_por_turma)

#### 3.4. Duração Média das Aulas
Se houver variações na duração, esta análise é útil.

In [ ]:
## 3.2. Carga Horária Planejada por Professor

# O cálculo de duração da aula é necessário para a agenda, pois o label_horario não tem o campo de duração
def calcular_duracao_agenda(start_str, end_str):
    start = datetime.strptime(start_str, '%H:%M')
    end = datetime.strptime(end_str, '%H:%M')
    # Trata caso de virada de dia (se necessário)
    if end < start:
        end += timedelta(days=1)
    return (end - start).total_seconds() / 3600 # Duração em Horas

# Aplica o cálculo no DataFrame de análise
df_analise['duracao_horas'] = df_analise.apply(
    lambda row: calcular_duracao_agenda(row['horario_inicio'], row['horario_fim']), 
    axis=1
)

carga_planejada_professor = df_analise.groupby('professor')['duracao_horas'].sum().round(2).reset_index()
carga_planejada_professor.columns = ['Professor', 'Carga_Horária_Planejada_Horas']

print("\n--- 👨‍🏫 3.2. Carga Horária Semanal Planejada por Professor ---")
print(carga_planejada_professor.sort_values(by='Carga_Horária_Planejada_Horas', ascending=False).to_string())

#### 4. Bônus: Identificação de Slots Completamente Livres (Avançado)
Para identificar horários que nunca tiveram aula em um período (ex: em todas as terças-feiras), precisaríamos de um intervalo de tempo completo de análise (seu ano letivo).

- Se você tiver o JSON do "ano letivo" e da "agenda semanal", podemos mapear os horários fixos de funcionamento da escola e fazer a subtração das aulas já ocupadas para obter os slots vazios com precisão.

Como alternativa, podemos identificar a intersecção dos horários vagos no dia mais livre.

In [ ]:
## 4. Bônus: Identificação de Slots Potenciais Vagos (Furos)

# 4.1. Mapeamento de todos os slots de tempo possíveis
horarios_disponiveis = dados_agenda_raw['config']['horario_id_para_label']

# 4.2. Identifica todas as turmas mapeadas que estão na agenda
turmas_na_agenda = df_agenda['turma_curta'].unique()

# 4.3. Cria todos os slots TURMA-DIA-HORARIO POSSÍVEIS (Cartesian Product)
slots_possiveis = []
for turma in turmas_na_agenda:
    for dia_int, dia_nome in mapa_dia_para_int.items():
        if dia_int < 5: # Considera apenas dias úteis (Seg-Sex)
            for horario_id, label_horario in horarios_disponiveis.items():
                start_time_str, _ = label_horario.split(' - ')
                slots_possiveis.append({
                    'turma_curta': turma,
                    'dia_semana': dia_int,
                    'dia_semana_nome': dia_nome,
                    'horario_inicio': start_time_str,
                    'label_horario': label_horario,
                    'chave_possivel': turma + '-' + str(dia_int) + '-' + start_time_str
                })

df_slots_possiveis = pd.DataFrame(slots_possiveis)

# 4.4. Cruza com os slots que estão realmente agendados
slots_agendados = set(df_agenda['chave_agendamento'])

df_slots_possiveis['agendado'] = df_slots_possiveis['chave_possivel'].apply(lambda x: x in slots_agendados)

# Filtra apenas os slots que não foram agendados (os "Furos")
furos_na_agenda = df_slots_possiveis[df_slots_possiveis['agendado'] == False]
furos_na_agenda = furos_na_agenda.sort_values(by=['turma_curta', 'dia_semana', 'horario_inicio'])

df_furos = furos_na_agenda[['turma_curta', 'dia_semana_nome', 'label_horario']]

print("\n--- 🎯 4. Slots Vagos na Agenda (Oportunidades de Agendamento) ---")
print("Estes são horários que poderiam ser agendados para as turmas, pois estão na grade horária padrão (config), mas não estão ocupados por uma disciplina na agenda fixa.")
print(df_furos.to_string())